In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji

     |████████████████████████████████| 170 kB 12.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=e82737d4cdbd277737f799d3138077ede1635fb60e46dc9c175d0d23285abd62
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 94638, done.
remote: Total 94638 (delta 0), reused 0 (delta 0), pack-reused 94638
Receiving objects: 100% (94638/94638), 79.84 MiB | 3.95 MiB/s, done.
Resolving deltas: 100% (68717/68717), done.


In [ ]:
%cd transformers

/content/transformers


In [ ]:
!pip3 install --upgrade .

Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 12.7 MB/s 
     |████████████████████████████████| 61 kB 434 kB/s 
     |████████████████████████████████| 895 kB 51.1 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
  Created wheel for transformers: filename=transformers-4.16.0.dev0-py3-none-any.whl size=3377405 sha256=8b6b51ea0f6a4f2b96bc3ee8a6b4f57c5bc4dbe6c2988534462922b88257c31f
  Stored in directory: /tmp/pip-ephem-wheel-c

In [ ]:
import os
import json
import random
import time
import datetime
import torch
import argparse
import numpy as np
import pandas as pd
from torch.nn import functional as F
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix
from transformers import get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler, Dataset

In [ ]:
batch_size = 4

In [ ]:
def convert_label(label):
    if label == "desirability":
        return 0
    elif label == "enabling context":
        return 1
    elif label == "can do":
        return 2
    elif label == "buzz":
        return 3
    elif label == "invitation":
        return 4
    else:
        raise Exception("label classes must be 'desirability' or 'enabling context' or 'buzz' or 'can do' or 'invitation'")

In [ ]:
%cd ..

/content


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Research Track/Dataset/Experiment Dataset/Dataset_V4.csv')
df

,Unnamed: 0,decision,created_at,id,id_str,full_text,source,retweet_count,favorite_count,reply,username_length,user_location
0,0,enabling context,2021-01-30 02:40:25+00:00,1355345090022154241,1355345090022154241,@science_barbie @priyology Cyanobacteria (also...,Twitter Web App,0,0,True,8,Unknown
1,1,enabling context,2021-01-30 01:05:16+00:00,1355321147701686273,1355321147701686273,New insights into coral bleaching by internati...,Twitter Web App,1,1,False,10,Unknown
2,2,buzz,2021-01-29 22:49:17+00:00,1355286925809094659,1355286925809094659,"Friday Tally:\n\n2 dives, continued coral blea...",Twitter for iPhone,0,1,False,11,Unknown
3,3,desirability,2021-01-29 22:09:20+00:00,1355276872410681348,1355276872410681348,Coral bleaching is so sad because they just be...,Twitter for iPhone,0,2,False,8,Unknown
4,4,desirability,2021-01-29 21:34:44+00:00,1355268165148815360,1355268165148815360,predicts that at the current rate of emissions...,Twitter Web App,0,0,True,13,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
1217,1217,buzz,2021-05-17 19:30:12+00:00,1394374712835264513,1394374712835264513,#ClimateChampions developed learning resources...,Twitter Web App,4,14,False,10,United States
1218,1218,desirability,2021-05-17 17:37:33+00:00,1394346363736018949,1394346363736018949,"""For all we hear about #coral bleaching &amp; ...",Twitter Web App,11,24,False,14,Unknown
1219,1219,enabling context,2021-05-17 17:01:39+00:00,1394337327930122250,1394337327930122250,Corals are critical. Track coral bleaching in ...,Hootsuite Inc.,10,29,False,13,United States
1220,1220,enabling context,2021-05-17 16:09:02+00:00,1394324089200779268,1394324089200779268,The number one root cause of coral reefs facin...,Twitter for iPhone,1,0,True,12,South Korea


In [ ]:
df = df[['decision','full_text']]
df = df.rename(columns={'full_text':'text','decision':'label'})

In [ ]:
df['label'] = df['label'].map({
    'desirability': 0, 
    'enabling context': 1,
    'can do': 2,
    'buzz':3,
    'invitation': 4
    })

In [ ]:
df['label'] = df['label'].astype(int)

In [ ]:
df

,label,text
0,1,@science_barbie @priyology Cyanobacteria (also...
1,1,New insights into coral bleaching by internati...
2,3,"Friday Tally:\n\n2 dives, continued coral blea..."
3,0,Coral bleaching is so sad because they just be...
4,0,predicts that at the current rate of emissions...
...,...,...
1217,3,#ClimateChampions developed learning resources...
1218,0,"""For all we hear about #coral bleaching &amp; ..."
1219,1,Corals are critical. Track coral bleaching in ...
1220,1,The number one root cause of coral reefs facin...


In [ ]:
def cleansing(text, stopword = None):
    # remove hastag
    text = ' '.join(word for word in text.split(' ') if not word.startswith('#'))
    # remove mention
    text = ' '.join(word for word in text.split(' ') if not word.startswith('@'))
    # remove URL
    text = ' '.join(word for word in text.split(' ') if not word.startswith('https:'))

    # case folding to lowercase
    text = text.lower()
    # split by space
    word_list = text.split()
    # pick only word with length greater than 2
    word_list = [word for word in word_list if len(word) > 2]
    # pick only alpha-numeric word 
    word_list = [word for word in word_list if word.isalnum()]

    if stopword == None:
        text = ' '.join(word_list)
    else:
        word_list = [word for word in word_list if word not in stopword]
        text = ' '.join(word_list)
                
    return text

In [ ]:
for i in range(len(df)):
    df.loc[i, 'text'] = cleansing(df.loc[i, 'text'])

In [ ]:
df

,label,text
0,1,cyanobacteria called are microorganisms that c...
1,1,new insights into coral bleaching internationa...
2,3,friday continued coral bleaching documentation...
3,0,coral bleaching sad because they just there vi...
4,0,predicts that the current rate there likely in...
...,...,...
1217,3,developed learning resources and curriculum fo...
1218,0,all hear about bleaching other early these imp...
1219,1,corals are track coral bleaching the pacific w...
1220,1,the number one root cause coral reefs facing d...


In [ ]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    for sentence in df[["text"]].values:
        sentence = sentence.item()
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      
                            add_special_tokens = True,  
                            max_length = 72,
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

In [ ]:
def prepare_dataloaders(test_df, batch_size):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    
    tweet_test = bert_encode(test_df, tokenizer)

    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    
    return test_dataloader

In [ ]:
test_dataloader = prepare_dataloaders(df, 1222)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
def test_encode(sentence):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

    encoded_dict = tokenizer.encode_plus(
                        sentence,                      
                        add_special_tokens = True,
                        max_length = 72,
                        pad_to_max_length = True,
                        truncation = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                )
           
    return encoded_dict['input_ids']

In [ ]:
def test_decode(tokens):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    return tokenizer.convert_ids_to_tokens(tokens)

In [ ]:
df.text[0]

'cyanobacteria called are microorganisms that can produce habs algal some which produce when released downstream the cause red and exacerbate coral'

In [ ]:
text_test = df.text[0]
text_encoded = test_encode(text_test)
text_decoded = test_decode(text_encoded[0, :130])


print(f'Shape      : {text_encoded.shape}')
print(f'Word Ids   : {text_encoded}')
print(f'Decoded Words   : {text_decoded}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Shape      : torch.Size([1, 72])
Word Ids   : tensor([[    0,  6936,   725,  4363,  1396, 40782,   488,    41,  8368, 45723,
            25,    56,  6052,   953,  1708,   638,  6355,   109,   248,  6052,
            64,  1549, 59266,     6,   311,  1007,    13,  1780, 47476, 42398,
         36507,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1]])
Decoded Words   : ['<s>', 'cy@@', 'an@@', 'ob@@', 'ac@@', 'teria', 'called', 'are', 'micro@@', 'organisms', 'that', 'can', 'produce', 'ha@@', 'bs', 'al@@', 'gal', 'some', 'which', 'produce', 'when', 'released', 'downstream', 'the', 'cause', 'red', 'and', 'ex@@', 'acer@@', 'bate', 'coral', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

In [ ]:
df['text']

0       cyanobacteria called are microorganisms that c...
1       new insights into coral bleaching internationa...
2       friday continued coral bleaching documentation...
3       coral bleaching sad because they just there vi...
4       predicts that the current rate there likely in...
                              ...                        
1217    developed learning resources and curriculum fo...
1218    all hear about bleaching other early these imp...
1219    corals are track coral bleaching the pacific w...
1220    the number one root cause coral reefs facing d...
1221    percent corals the are risk vanishing unless p...
Name: text, Length: 1222, dtype: object

In [ ]:
from transformers import RobertaConfig, RobertaModel, BertPreTrainedModel
from torch.nn import CrossEntropyLoss
from torch import nn
from typing import Tuple
import torch
!wget https://public.vinai.io/BERTweet_base_transformers.tar.gz

!tar -xzvf BERTweet_base_transformers.tar.gz

class BERTweetModelForClassification(BertPreTrainedModel):
    base_model_prefix = "roberta"

    def __init__(self):
        self.num_labels: int = 5
        config: RobertaConfig = RobertaConfig.from_pretrained(
            "vinai/bertweet-base",
            output_hidden_states=True,
        )
        super().__init__(config)
        self.model: RobertaModel = RobertaModel.from_pretrained(
            "vinai/bertweet-base",
            config=config
        )
        self.dense = nn.Linear(in_features=768 * 1,
                               out_features=768,
                               )
        self.dropout = nn.Dropout(p=0.1)
        self.dense_2 = nn.Linear(in_features=768,
                                 out_features=256,
                                 )
        self.classifier = nn.Linear(in_features=256,
                                    out_features=self.num_labels,
                                    )

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
    ):
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
        )
        # Take <CLS> token for Native Layer Norm Backward
        hidden_states: [torch.tensor] = outputs[2]
        last_sequence_output: torch.tensor = hidden_states[-1][:, 0, :]

        sequence_output: torch.tensor = torch.cat((
            last_sequence_output,
        ), dim=1)

        sequence_output = self.dense(sequence_output)
        sequence_output = self.dropout(sequence_output)
        sequence_output = self.dense_2(sequence_output)
        sequence_output = self.dropout(sequence_output)

        logits: torch.tensor = self.classifier(sequence_output)
        return outputs  # loss, logits

--2021-12-29 11:17:28--  https://public.vinai.io/BERTweet_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 52.222.149.57, 52.222.149.21, 52.222.149.126, ...
Connecting to public.vinai.io (public.vinai.io)|52.222.149.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322076118 (307M) [application/x-tar]
Saving to: ‘BERTweet_base_transformers.tar.gz’

BERTweet_base_trans 100%[===================>] 307.16M   193MB/s    in 1.6s    

2021-12-29 11:17:30 (193 MB/s) - ‘BERTweet_base_transformers.tar.gz’ saved [322076118/322076118]

BERTweet_base_transformers/
BERTweet_base_transformers/config.json
BERTweet_base_transformers/bpe.codes
BERTweet_base_transformers/model.bin
BERTweet_base_transformers/dict.txt


In [ ]:
def test(model,test_dataloader):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    preds = []
    for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
    
        with torch.no_grad():        
            outputs = model(b_input_ids,  
                            attention_mask=b_input_mask,)
            
        logits = outputs[2]
        preds.append(logits)
        
    return preds

In [ ]:
model = BERTweetModelForClassification()
embed = test(model,test_dataloader)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(len(embed[0][0]))

1222


In [ ]:

a = np.array(embed[0][0].detach().cpu().numpy())
# b = np.array(embed[0][0][0].detach().cpu().numpy())

In [ ]:
print(a[1])

[[ 0.03415883 -0.0413883  -0.00334099 ... -0.00137679  0.00576577
  -0.01988301]
 [-0.4235622   0.01526384 -0.14190997 ... -0.23943411  0.47758532
  -0.0670002 ]
 [ 0.33704147  0.3330365  -0.395935   ... -0.05097864  0.15584959
  -0.2764096 ]
 ...
 [-0.17921612  0.07901301 -0.04808439 ... -0.05727765 -0.00495946
   0.02153439]
 [-0.17921612  0.07901301 -0.04808439 ... -0.05727765 -0.00495946
   0.02153439]
 [-0.17921612  0.07901301 -0.04808439 ... -0.05727765 -0.00495946
   0.02153439]]


In [ ]:
df_avg = []
for i in range(0,len(a)):
  df_avg.append(a[i])

In [ ]:
print(len(df_avg))

1222


In [ ]:
df_average = []
for i in range(0,len(df_avg)):
  df_average.append(df_avg[i].mean(axis=0))

In [ ]:
from pandas import DataFrame

new_df = DataFrame(df_average)
new_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.098396,0.033388,-0.027194,0.027376,-0.053213,-0.229573,-0.064048,-0.019447,-0.100565,0.041436,0.055234,-0.112004,-0.026260,-0.003475,0.010007,-0.019481,0.101861,-0.012609,0.173384,-0.085550,-0.029446,0.029432,-0.030919,0.035403,0.010309,-0.039743,0.033807,0.025733,-0.048577,0.017068,-0.015503,-0.123276,-0.010651,-0.023103,-0.082272,0.100179,-0.016894,-0.020930,-0.033845,0.002016,...,-0.072309,0.012263,0.072801,-0.018215,0.010691,-0.009838,0.044008,-0.001602,0.001398,0.030275,-0.060219,-0.065032,0.015660,0.011598,-0.040531,-0.099422,-0.072954,-0.047893,-0.072633,-0.010661,-0.112072,0.021292,0.094422,-0.020142,0.030179,-0.145116,-0.076539,0.010781,0.051880,0.000173,-0.034359,-0.114203,-0.010197,0.004564,0.019928,-0.104868,-0.034168,-0.045843,0.038660,-0.000471
1,-0.120705,0.053524,-0.011742,-0.024066,-0.020597,-0.164896,-0.043475,-0.023146,-0.129938,0.087002,0.028965,-0.146307,-0.019684,-0.015311,0.044594,-0.024534,0.073238,-0.047343,0.179670,-0.063624,-0.000370,0.007219,-0.078712,0.020497,0.035993,-0.027106,0.003012,0.053134,-0.083262,0.013684,-0.076660,-0.083257,0.020718,0.009445,-0.108861,0.056674,-0.060201,-0.048310,-0.058839,0.054120,...,-0.093053,-0.013857,0.104834,-0.016492,0.052866,0.002449,0.065856,0.004242,-0.005995,0.044031,-0.041887,-0.096008,-0.020696,-0.005225,-0.017845,-0.045892,-0.116985,-0.096626,-0.148036,0.015681,-0.120225,0.021378,0.028131,0.010330,-0.001928,-0.113713,-0.104708,0.016720,0.055613,-0.011071,-0.021663,-0.129373,0.012325,0.032039,-0.004675,-0.115931,-0.069920,-0.016818,0.004227,0.011620
2,-0.122572,0.035299,-0.054835,-0.016522,-0.035902,-0.237163,-0.053316,-0.018964,-0.095764,0.097162,0.024912,-0.140865,-0.007761,-0.022813,0.016860,-0.040402,0.074898,-0.016770,0.194662,-0.055484,0.006319,0.061244,-0.083421,0.071217,0.051419,0.004833,0.012157,0.036231,-0.054166,0.000149,-0.033718,-0.113663,-0.023804,0.013130,-0.104261,0.067874,-0.065606,-0.047905,-0.041829,0.017031,...,-0.097423,0.023294,0.086937,-0.071755,0.037257,0.019510,0.031695,-0.012924,-0.035026,0.032390,-0.106572,-0.154233,-0.002395,0.003598,-0.057729,-0.058876,-0.098557,-0.094550,-0.112756,0.015281,-0.166485,0.038607,0.104973,-0.045032,0.016939,-0.165245,-0.093023,0.009059,0.064336,-0.026907,-0.010531,-0.135643,-0.019771,0.003220,0.034839,-0.120211,-0.102284,-0.035232,0.025805,0.012726
3,-0.146215,0.053986,-0.055207,0.015340,-0.009471,-0.204446,-0.058692,-0.054176,-0.126879,0.080218,0.048536,-0.139596,-0.028624,-0.014139,0.028696,-0.036240,0.061228,-0.028641,0.175150,-0.045821,0.029241,0.067196,-0.015185,0.085199,0.017097,-0.000817,0.006146,0.056145,-0.100741,0.006668,-0.021476,-0.142058,-0.028602,0.001919,-0.108112,0.091110,-0.041157,-0.014675,-0.040719,0.059271,...,-0.076370,0.020831,0.099043,-0.046696,0.048050,0.009879,0.019632,-0.016999,-0.016689,0.054947,-0.078176,-0.061277,0.029513,-0.022962,-0.062379,-0.087180,-0.050885,-0.083744,-0.100700,-0.008816,-0.124461,0.038384,0.086821,-0.072922,0.035840,-0.136714,-0.088404,0.030295,0.022566,0.014406,-0.001857,-0.128572,-0.004951,-0.003194,0.040886,-0.120653,-0.097649,-0.021897,0.017753,0.027179
4,-0.102229,0.067026,0.019315,0.038599,-0.027093,-0.219828,-0.011722,0.001191,-0.102072,0.089824,-0.006155,-0.102504,-0.031258,-0.031609,0.039617,-0.053095,0.118546,-0.058241,0.124888,-0.056428,-0.015225,0.049737,0.006300,0.034104,-0.002576,-0.041102,-0.019003,0.031235,-0.016504,-0.003390,-0.025735,-0.107479,-0.009990,0.024677,-0.067040,0.023947,-0.045911,-0.021708,-0.040793,0.050309,...,-0.037452,0.056382,0.065883,0.000475,0.007120,-0.036393,0.043222,-0.025028,-0.017490,0.042263,-0.064573,-0.115126,0.019282,-0.020778,-0.034549,-0.030730,-0.032842,-0.094107,-0.093373,0.002244,-0.123450,0.048515,0.049222,-0.059212,0.010422,-0

In [ ]:
df_final = pd.concat([df,new_df],axis=1)

In [ ]:
df_final

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,1,cyanobacteria called are microorganisms that c...,-0.098396,0.033388,-0.027194,0.027376,-0.053213,-0.229573,-0.064048,-0.019447,-0.100565,0.041436,0.055234,-0.112004,-0.026260,-0.003475,0.010007,-0.019481,0.101861,-0.012609,0.173384,-0.085550,-0.029446,0.029432,-0.030919,0.035403,0.010309,-0.039743,0.033807,0.025733,-0.048577,0.017068,-0.015503,-0.123276,-0.010651,-0.023103,-0.082272,0.100179,-0.016894,-0.020930,...,-0.072309,0.012263,0.072801,-0.018215,0.010691,-0.009838,0.044008,-0.001602,0.001398,0.030275,-0.060219,-0.065032,0.015660,0.011598,-0.040531,-0.099422,-0.072954,-0.047893,-0.072633,-0.010661,-0.112072,0.021292,0.094422,-0.020142,0.030179,-0.145116,-0.076539,0.010781,0.051880,0.000173,-0.034359,-0.114203,-0.010197,0.004564,0.019928,-0.104868,-0.034168,-0.045843,0.038660,-0.000471
1,1,new insights into coral bleaching internationa...,-0.120705,0.053524,-0.011742,-0.024066,-0.020597,-0.164896,-0.043475,-0.023146,-0.129938,0.087002,0.028965,-0.146307,-0.019684,-0.015311,0.044594,-0.024534,0.073238,-0.047343,0.179670,-0.063624,-0.000370,0.007219,-0.078712,0.020497,0.035993,-0.027106,0.003012,0.053134,-0.083262,0.013684,-0.076660,-0.083257,0.020718,0.009445,-0.108861,0.056674,-0.060201,-0.048310,...,-0.093053,-0.013857,0.104834,-0.016492,0.052866,0.002449,0.065856,0.004242,-0.005995,0.044031,-0.041887,-0.096008,-0.020696,-0.005225,-0.017845,-0.045892,-0.116985,-0.096626,-0.148036,0.015681,-0.120225,0.021378,0.028131,0.010330,-0.001928,-0.113713,-0.104708,0.016720,0.055613,-0.011071,-0.021663,-0.129373,0.012325,0.032039,-0.004675,-0.115931,-0.069920,-0.016818,0.004227,0.011620
2,3,friday continued coral bleaching documentation...,-0.122572,0.035299,-0.054835,-0.016522,-0.035902,-0.237163,-0.053316,-0.018964,-0.095764,0.097162,0.024912,-0.140865,-0.007761,-0.022813,0.016860,-0.040402,0.074898,-0.016770,0.194662,-0.055484,0.006319,0.061244,-0.083421,0.071217,0.051419,0.004833,0.012157,0.036231,-0.054166,0.000149,-0.033718,-0.113663,-0.023804,0.013130,-0.104261,0.067874,-0.065606,-0.047905,...,-0.097423,0.023294,0.086937,-0.071755,0.037257,0.019510,0.031695,-0.012924,-0.035026,0.032390,-0.106572,-0.154233,-0.002395,0.003598,-0.057729,-0.058876,-0.098557,-0.094550,-0.112756,0.015281,-0.166485,0.038607,0.104973,-0.045032,0.016939,-0.165245,-0.093023,0.009059,0.064336,-0.026907,-0.010531,-0.135643,-0.019771,0.003220,0.034839,-0.120211,-0.102284,-0.035232,0.025805,0.012726
3,0,coral bleaching sad because they just there vi...,-0.146215,0.053986,-0.055207,0.015340,-0.009471,-0.204446,-0.058692,-0.054176,-0.126879,0.080218,0.048536,-0.139596,-0.028624,-0.014139,0.028696,-0.036240,0.061228,-0.028641,0.175150,-0.045821,0.029241,0.067196,-0.015185,0.085199,0.017097,-0.000817,0.006146,0.056145,-0.100741,0.006668,-0.021476,-0.142058,-0.028602,0.001919,-0.108112,0.091110,-0.041157,-0.014675,...,-0.076370,0.020831,0.099043,-0.046696,0.048050,0.009879,0.019632,-0.016999,-0.016689,0.054947,-0.078176,-0.061277,0.029513,-0.022962,-0.062379,-0.087180,-0.050885,-0.083744,-0.100700,-0.008816,-0.124461,0.038384,0.086821,-0.072922,0.035840,-0.136714,-0.088404,0.030295,0.022566,0.014406,-0.001857,-0.128572,-0.004951,-0.003194,0.040886,-0.120653,-0.097649,-0.021897,0.017753,0.027179
4,0,predicts that the current rate there likely in...,-0.102229,0.067026,0.019315,0.038599,-0.027093,-0.219828,-0.011722,0.001191,-0.102072,0.089824,-0.006155,-0.102504,-0.031258,-0.031609,0.039617,-0.053095,0.118546,-0.058241,0.124888,-0.056428,-0.015225,0.049737,0.006300,0.034104,-0.002576,-0.041102,-0.019003,0.031235,-0.016504,-0.003390,-0.025735,-0.107479,-0.009990,0.024677,-0.067040,0.023947,-0.045911,-0.021708,...,-0.037452,0.056382,0.065883,0.000475,0.007120,-0.036393,0.043222,-0.025

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 5):

    sentences = df_final['text']

    X = df_average

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    df_final['cluster'] = pd.Series(assigned_clusters, index=data.index)
    df_final['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

In [ ]:
clustering_question(df_final)

(      label  ...                                           centroid
 0         1  ...  [-0.11269804, 0.041881617, -0.02640572, 0.0012...
 1         1  ...  [-0.12596914, 0.047390807, -0.04053343, -0.006...
 2         3  ...  [-0.12596914, 0.047390807, -0.04053343, -0.006...
 3         0  ...  [-0.11269804, 0.041881617, -0.02640572, 0.0012...
 4         0  ...  [-0.10183591, 0.033912547, -0.017202158, 0.005...
 ...     ...  ...                                                ...
 1217      3  ...  [-0.12596914, 0.047390807, -0.04053343, -0.006...
 1218      0  ...  [-0.12596914, 0.047390807, -0.04053343, -0.006...
 1219      1  ...  [-0.12596914, 0.047390807, -0.04053343, -0.006...
 1220      1  ...  [-0.11269804, 0.041881617, -0.02640572, 0.0012...
 1221      1  ...  [-0.11269804, 0.041881617, -0.02640572, 0.0012...
 
 [1222 rows x 772 columns],
 [2,
  1,
  1,
  2,
  3,
  3,
  0,
  1,
  1,
  4,
  0,
  1,
  0,
  0,
  3,
  1,
  1,
  0,
  2,
  2,
  3,
  0,
  0,
  0,
  0,
  0,
  2,
  3,
  

In [ ]:
df_final

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
0,1,cyanobacteria called are microorganisms that c...,-0.098396,0.033388,-0.027194,0.027376,-0.053213,-0.229573,-0.064048,-0.019447,-0.100565,0.041436,0.055234,-0.112004,-0.026260,-0.003475,0.010007,-0.019481,0.101861,-0.012609,0.173384,-0.085550,-0.029446,0.029432,-0.030919,0.035403,0.010309,-0.039743,0.033807,0.025733,-0.048577,0.017068,-0.015503,-0.123276,-0.010651,-0.023103,-0.082272,0.100179,-0.016894,-0.020930,...,0.072801,-0.018215,0.010691,-0.009838,0.044008,-0.001602,0.001398,0.030275,-0.060219,-0.065032,0.015660,0.011598,-0.040531,-0.099422,-0.072954,-0.047893,-0.072633,-0.010661,-0.112072,0.021292,0.094422,-0.020142,0.030179,-0.145116,-0.076539,0.010781,0.051880,0.000173,-0.034359,-0.114203,-0.010197,0.004564,0.019928,-0.104868,-0.034168,-0.045843,0.038660,-0.000471,2,"[-0.11269804, 0.041881617, -0.02640572, 0.0012..."
1,1,new insights into coral bleaching internationa...,-0.120705,0.053524,-0.011742,-0.024066,-0.020597,-0.164896,-0.043475,-0.023146,-0.129938,0.087002,0.028965,-0.146307,-0.019684,-0.015311,0.044594,-0.024534,0.073238,-0.047343,0.179670,-0.063624,-0.000370,0.007219,-0.078712,0.020497,0.035993,-0.027106,0.003012,0.053134,-0.083262,0.013684,-0.076660,-0.083257,0.020718,0.009445,-0.108861,0.056674,-0.060201,-0.048310,...,0.104834,-0.016492,0.052866,0.002449,0.065856,0.004242,-0.005995,0.044031,-0.041887,-0.096008,-0.020696,-0.005225,-0.017845,-0.045892,-0.116985,-0.096626,-0.148036,0.015681,-0.120225,0.021378,0.028131,0.010330,-0.001928,-0.113713,-0.104708,0.016720,0.055613,-0.011071,-0.021663,-0.129373,0.012325,0.032039,-0.004675,-0.115931,-0.069920,-0.016818,0.004227,0.011620,1,"[-0.12596914, 0.047390807, -0.04053343, -0.006..."
2,3,friday continued coral bleaching documentation...,-0.122572,0.035299,-0.054835,-0.016522,-0.035902,-0.237163,-0.053316,-0.018964,-0.095764,0.097162,0.024912,-0.140865,-0.007761,-0.022813,0.016860,-0.040402,0.074898,-0.016770,0.194662,-0.055484,0.006319,0.061244,-0.083421,0.071217,0.051419,0.004833,0.012157,0.036231,-0.054166,0.000149,-0.033718,-0.113663,-0.023804,0.013130,-0.104261,0.067874,-0.065606,-0.047905,...,0.086937,-0.071755,0.037257,0.019510,0.031695,-0.012924,-0.035026,0.032390,-0.106572,-0.154233,-0.002395,0.003598,-0.057729,-0.058876,-0.098557,-0.094550,-0.112756,0.015281,-0.166485,0.038607,0.104973,-0.045032,0.016939,-0.165245,-0.093023,0.009059,0.064336,-0.026907,-0.010531,-0.135643,-0.019771,0.003220,0.034839,-0.120211,-0.102284,-0.035232,0.025805,0.012726,1,"[-0.12596914, 0.047390807, -0.04053343, -0.006..."
3,0,coral bleaching sad because they just there vi...,-0.146215,0.053986,-0.055207,0.015340,-0.009471,-0.204446,-0.058692,-0.054176,-0.126879,0.080218,0.048536,-0.139596,-0.028624,-0.014139,0.028696,-0.036240,0.061228,-0.028641,0.175150,-0.045821,0.029241,0.067196,-0.015185,0.085199,0.017097,-0.000817,0.006146,0.056145,-0.100741,0.006668,-0.021476,-0.142058,-0.028602,0.001919,-0.108112,0.091110,-0.041157,-0.014675,...,0.099043,-0.046696,0.048050,0.009879,0.019632,-0.016999,-0.016689,0.054947,-0.078176,-0.061277,0.029513,-0.022962,-0.062379,-0.087180,-0.050885,-0.083744,-0.100700,-0.008816,-0.124461,0.038384,0.086821,-0.072922,0.035840,-0.136714,-0.088404,0.030295,0.022566,0.014406,-0.001857,-0.128572,-0.004951,-0.003194,0.040886,-0.120653,-0.097649,-0.021897,0.017753,0.027179,2,"[-0.11269804, 0.041881617, -0.02640572, 0.0012..."
4,0,predicts that the current rate there likely in...,-0.102229,0.067026,0.019315,0.038599,-0.027093,-0.219828,-0.011722,0.001191,-0.102072,0.089824,-0.006155,-0.102504,-0.031258,-0.031609,0.039617,-0.053095,0.118546,-0.058241,0.124888,-0.056428,-0.015225,0.049737,0.006300,0.034104,-0.002576,-0.041102,-0.019003,0.031235,-0.016504,-0.003390,-0.02

**Penentuan Cluster Zero**

In [ ]:
cluster_zero = df_final.loc[df_final['cluster'] == 0]
cluster_zero

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
6,1,happening coral bleaching,-0.150780,0.061683,-0.067028,-0.022703,-0.030862,-0.263424,-0.072246,-0.068440,-0.169194,0.119516,0.047721,-0.198146,-0.015736,-0.031108,0.012453,-0.063242,0.077202,0.003690,0.238851,-0.096328,0.009190,0.069067,-0.095525,0.062964,0.073451,-0.011285,0.012657,0.114592,-0.139249,0.005060,-0.052932,-0.209811,-0.064030,0.007778,-0.165941,0.138745,-0.107136,-0.045610,...,0.122691,-0.066091,0.021992,0.000578,0.022321,-0.032555,-0.062208,0.068207,-0.143546,-0.163030,-0.012477,-0.047540,-0.017388,-0.094336,-0.145541,-0.160565,-0.171757,-0.019501,-0.202113,0.036106,0.122147,-0.045722,-0.008991,-0.207497,-0.124164,0.007823,0.099426,-0.024385,-0.011416,-0.175152,-0.014522,0.031894,0.008535,-0.118016,-0.138212,-0.060827,-0.016423,0.015755,0,"[-0.14588952, 0.061341245, -0.05078541, -0.014..."
10,1,research reveals genetic response ocean warmin...,-0.127385,0.069656,-0.052226,-0.006350,-0.032662,-0.247373,-0.042024,-0.047815,-0.168456,0.121043,0.029410,-0.189005,-0.006727,-0.043273,0.015077,-0.065500,0.087884,-0.005767,0.227192,-0.070487,0.005033,0.055284,-0.075216,0.059135,0.062674,-0.027728,0.018637,0.097157,-0.116495,0.016350,-0.059991,-0.169883,-0.028831,0.017451,-0.134492,0.122338,-0.104334,-0.050590,...,0.101117,-0.060252,0.027417,-0.010272,0.018430,-0.020804,-0.054661,0.044165,-0.130839,-0.148435,0.009448,-0.039367,-0.042589,-0.050704,-0.105218,-0.155731,-0.167868,-0.023796,-0.178962,0.058342,0.075603,-0.034453,-0.005651,-0.166683,-0.130260,0.003777,0.092903,-0.019163,0.007374,-0.157931,-0.024535,0.017527,0.009921,-0.120410,-0.128447,-0.062182,-0.006091,0.041649,0,"[-0.14588952, 0.061341245, -0.05078541, -0.014..."
12,1,american lobster postlarvae alter gene regulat...,-0.135246,0.067846,-0.044082,0.008454,-0.029045,-0.237102,-0.030546,-0.061824,-0.145432,0.118600,0.039771,-0.175610,-0.010589,-0.034135,0.023035,-0.066663,0.090407,-0.015510,0.218923,-0.081997,0.005125,0.029748,-0.063002,0.037221,0.067188,-0.020946,0.017823,0.086114,-0.109712,0.019987,-0.053141,-0.166376,-0.036970,-0.003883,-0.129344,0.126454,-0.084961,-0.054472,...,0.113274,-0.063300,0.033202,-0.009422,0.036986,-0.024419,-0.051786,0.033852,-0.120876,-0.115695,0.022465,-0.039783,-0.035252,-0.059480,-0.088165,-0.139759,-0.155146,-0.023935,-0.164651,0.064538,0.078695,-0.047390,0.002869,-0.148283,-0.116972,0.002545,0.096802,-0.022235,0.015365,-0.154373,-0.017605,0.026378,0.008838,-0.132048,-0.121265,-0.039013,0.001596,0.056974,0,"[-0.14588952, 0.061341245, -0.05078541, -0.014..."
13,0,even minecraft reefs safe from the threat cora...,-0.137318,0.053263,-0.068685,-0.006250,-0.039266,-0.260417,-0.050190,-0.038220,-0.137594,0.098268,0.027944,-0.191580,-0.025952,-0.020379,0.028093,-0.053566,0.068843,-0.002003,0.244606,-0.079941,0.012894,0.082348,-0.085129,0.076999,0.087570,-0.012291,-0.002845,0.102127,-0.114529,0.006490,-0.041300,-0.203519,-0.028527,-0.004907,-0.146890,0.123968,-0.095587,-0.038648,...,0.117739,-0.073787,0.009095,-0.000226,0.019500,-0.029549,-0.044759,0.070318,-0.126194,-0.150628,-0.003293,-0.025470,-0.030766,-0.081429,-0.111610,-0.139452,-0.150649,-0.008509,-0.190814,0.045995,0.105007,-0.047074,0.022725,-0.198164,-0.094973,0.005043,0.093707,-0.026404,-0.016220,-0.153669,-0.017428,0.018972,-0.007178,-0.120740,-0.109472,-0.054576,-0.006987,0.007008,0,"[-0.14588952, 0.061341245, -0.05078541, -0.014..."
17,1,shelly the ongoing problem coral bleaching som...,-0.131984,0.044261,-0.047275,0.007298,-0.058001,-0.253160,-0.055114,-0.055821,-0.139098,0.114143,0.023454,-0.186467,-0.010292,-0.014901,0.000447,-0.052254,0.079929,-0.000578,0.196911,-0.080763,-0.002858,0.054906,-0.071407,0.072346,0.057831,-0.013464,0.004719,0.112812,-0.126924,-0.004512,-0.048602,-

In [ ]:
cluster_zero['label'].value_counts()
# cluster 0 = Cluster Enabling Context

1    147
0    111
3     83
4     23
2     19
Name: label, dtype: int64

**Penentuan Cluster One**

In [ ]:
cluster_one = df_final.loc[df_final['cluster'] == 1]
cluster_one

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
1,1,new insights into coral bleaching internationa...,-0.120705,0.053524,-0.011742,-0.024066,-0.020597,-0.164896,-0.043475,-0.023146,-0.129938,0.087002,0.028965,-0.146307,-0.019684,-0.015311,0.044594,-0.024534,0.073238,-0.047343,0.179670,-0.063624,-0.000370,0.007219,-0.078712,0.020497,0.035993,-0.027106,0.003012,0.053134,-0.083262,0.013684,-0.076660,-0.083257,0.020718,0.009445,-0.108861,0.056674,-0.060201,-0.048310,...,0.104834,-0.016492,0.052866,0.002449,0.065856,0.004242,-0.005995,0.044031,-0.041887,-0.096008,-0.020696,-0.005225,-0.017845,-0.045892,-0.116985,-0.096626,-0.148036,0.015681,-0.120225,0.021378,0.028131,0.010330,-0.001928,-0.113713,-0.104708,0.016720,0.055613,-0.011071,-0.021663,-0.129373,0.012325,0.032039,-0.004675,-0.115931,-0.069920,-0.016818,0.004227,0.011620,1,"[-0.12596914, 0.047390807, -0.04053343, -0.006..."
2,3,friday continued coral bleaching documentation...,-0.122572,0.035299,-0.054835,-0.016522,-0.035902,-0.237163,-0.053316,-0.018964,-0.095764,0.097162,0.024912,-0.140865,-0.007761,-0.022813,0.016860,-0.040402,0.074898,-0.016770,0.194662,-0.055484,0.006319,0.061244,-0.083421,0.071217,0.051419,0.004833,0.012157,0.036231,-0.054166,0.000149,-0.033718,-0.113663,-0.023804,0.013130,-0.104261,0.067874,-0.065606,-0.047905,...,0.086937,-0.071755,0.037257,0.019510,0.031695,-0.012924,-0.035026,0.032390,-0.106572,-0.154233,-0.002395,0.003598,-0.057729,-0.058876,-0.098557,-0.094550,-0.112756,0.015281,-0.166485,0.038607,0.104973,-0.045032,0.016939,-0.165245,-0.093023,0.009059,0.064336,-0.026907,-0.010531,-0.135643,-0.019771,0.003220,0.034839,-0.120211,-0.102284,-0.035232,0.025805,0.012726,1,"[-0.12596914, 0.047390807, -0.04053343, -0.006..."
7,4,going the biggest coral restoration event happ...,-0.136090,0.045172,-0.045797,0.017917,-0.030351,-0.215070,-0.045282,-0.046779,-0.143029,0.112204,0.025480,-0.146856,-0.005865,-0.028014,0.017531,-0.048131,0.085106,-0.012866,0.188264,-0.039521,-0.021785,0.071766,-0.055351,0.069331,0.067731,0.003049,0.007481,0.086661,-0.108375,0.010037,-0.059218,-0.161624,-0.030795,0.000085,-0.106233,0.126473,-0.066664,-0.017764,...,0.085739,-0.036682,0.015747,0.009138,0.058285,-0.067776,-0.033243,0.048986,-0.120369,-0.141847,-0.020153,-0.030377,-0.039700,-0.104660,-0.088682,-0.124597,-0.143791,-0.009429,-0.130015,0.027469,0.064692,-0.002315,0.022588,-0.146447,-0.094037,0.014371,0.091316,-0.019459,0.020663,-0.132509,-0.023042,-0.000919,0.061451,-0.138151,-0.090708,-0.000124,0.018955,0.032082,1,"[-0.12596914, 0.047390807, -0.04053343, -0.006..."
8,4,january 30th 2021 guest lecture franziska ston...,-0.106663,0.073373,-0.032788,0.026681,-0.028721,-0.198818,-0.036388,-0.064726,-0.135827,0.084047,0.003794,-0.153852,0.029289,-0.028381,0.036133,-0.030916,0.070800,0.003256,0.209444,-0.074536,0.018779,0.015489,-0.032328,0.002101,0.052031,-0.004431,0.017569,0.040465,-0.085649,-0.017382,-0.040629,-0.128538,-0.026079,-0.013760,-0.106606,0.089823,-0.054089,-0.027910,...,0.031645,-0.002706,0.017650,-0.001579,0.030897,-0.005001,-0.035576,0.012358,-0.103164,-0.138568,-0.009985,0.005192,-0.037628,-0.104381,-0.095227,-0.136031,-0.081098,0.013630,-0.142169,0.021863,0.080143,-0.003268,0.016500,-0.135262,-0.092984,0.057724,0.044958,-0.037698,0.018383,-0.113753,-0.027438,0.015815,0.011029,-0.093247,-0.076871,-0.016689,-0.021296,0.010157,1,"[-0.12596914, 0.047390807, -0.04053343, -0.006..."
11,3,going special dive today with the coral restor...,-0.111306,0.003214,-0.032490,0.013892,-0.014348,-0.213166,-0.049088,-0.018492,-0.135484,0.067287,0.022991,-0.127049,0.012118,-0.010342,0.009048,-0.051852,0.082429,-0.020160,0.167826,-0.069569,-0.015790,0.049655,-0.037349,0.070132,0.047870,0.008588,-0.012677,0.055566,-0.078919,-0.007635,-0.07

In [ ]:
cluster_one['label'].value_counts()
#Cluster 1 = Cluster Enabling Context

1    113
3     98
0     83
4     17
2      8
Name: label, dtype: int64

**Penentuan Cluster Two**




In [ ]:
cluster_two = df_final.loc[df_final['cluster'] == 2]
cluster_two

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
0,1,cyanobacteria called are microorganisms that c...,-0.098396,0.033388,-0.027194,0.027376,-0.053213,-0.229573,-0.064048,-0.019447,-0.100565,0.041436,0.055234,-0.112004,-0.026260,-0.003475,0.010007,-0.019481,0.101861,-0.012609,0.173384,-0.085550,-0.029446,0.029432,-0.030919,0.035403,0.010309,-0.039743,0.033807,0.025733,-0.048577,0.017068,-0.015503,-0.123276,-0.010651,-0.023103,-0.082272,0.100179,-0.016894,-0.020930,...,0.072801,-0.018215,0.010691,-0.009838,0.044008,-0.001602,0.001398,0.030275,-0.060219,-0.065032,0.015660,0.011598,-0.040531,-0.099422,-0.072954,-0.047893,-0.072633,-0.010661,-0.112072,0.021292,0.094422,-0.020142,0.030179,-0.145116,-0.076539,0.010781,0.051880,0.000173,-0.034359,-0.114203,-0.010197,0.004564,0.019928,-0.104868,-0.034168,-0.045843,0.038660,-0.000471,2,"[-0.11269804, 0.041881617, -0.02640572, 0.0012..."
3,0,coral bleaching sad because they just there vi...,-0.146215,0.053986,-0.055207,0.015340,-0.009471,-0.204446,-0.058692,-0.054176,-0.126879,0.080218,0.048536,-0.139596,-0.028624,-0.014139,0.028696,-0.036240,0.061228,-0.028641,0.175150,-0.045821,0.029241,0.067196,-0.015185,0.085199,0.017097,-0.000817,0.006146,0.056145,-0.100741,0.006668,-0.021476,-0.142058,-0.028602,0.001919,-0.108112,0.091110,-0.041157,-0.014675,...,0.099043,-0.046696,0.048050,0.009879,0.019632,-0.016999,-0.016689,0.054947,-0.078176,-0.061277,0.029513,-0.022962,-0.062379,-0.087180,-0.050885,-0.083744,-0.100700,-0.008816,-0.124461,0.038384,0.086821,-0.072922,0.035840,-0.136714,-0.088404,0.030295,0.022566,0.014406,-0.001857,-0.128572,-0.004951,-0.003194,0.040886,-0.120653,-0.097649,-0.021897,0.017753,0.027179,2,"[-0.11269804, 0.041881617, -0.02640572, 0.0012..."
18,1,wonderful see cover current sea anemone tentac...,-0.124976,0.026426,-0.020819,0.033390,-0.019896,-0.243807,-0.046268,-0.034371,-0.100862,0.087281,0.033676,-0.068599,0.002384,-0.008481,0.030387,-0.017942,0.095340,-0.038072,0.142257,-0.031948,0.027240,0.029435,-0.062045,0.052000,0.023902,0.007808,0.007162,0.035111,-0.080529,0.046693,-0.020189,-0.109403,0.012465,0.009661,-0.109164,0.076022,-0.037622,-0.010827,...,0.093505,-0.023318,0.013033,0.047417,0.035977,0.003073,-0.044110,0.030487,-0.080972,-0.086351,0.000317,-0.033830,-0.032456,-0.062265,-0.067824,-0.107498,-0.056328,-0.034923,-0.118212,-0.014676,0.060679,-0.044705,0.023209,-0.087520,-0.032401,0.048758,0.074402,0.029335,-0.013056,-0.108879,-0.022011,-0.018496,0.036934,-0.112154,-0.046998,-0.050288,0.036574,-0.015409,2,"[-0.11269804, 0.041881617, -0.02640572, 0.0012..."
19,1,ocean warming may exacerbate their effects spe...,-0.126710,0.052468,-0.013383,-0.006734,-0.047741,-0.173739,-0.057727,-0.014264,-0.103970,0.096858,0.030761,-0.134055,-0.027194,-0.021837,0.032779,-0.027743,0.119888,-0.025683,0.144789,-0.047601,-0.008617,0.030200,0.000023,0.023240,0.053037,-0.013831,0.015417,0.006571,-0.051598,-0.005041,-0.053218,-0.141513,0.028380,0.016070,-0.046635,0.082834,-0.067051,-0.018268,...,0.048063,-0.022539,0.036543,-0.027839,0.048913,0.002731,-0.011967,0.002541,-0.069904,-0.138551,0.055514,0.006827,-0.019433,-0.024367,-0.056351,-0.107223,-0.114611,-0.017217,-0.094294,0.021830,0.018521,-0.043717,-0.008935,-0.162722,-0.043254,0.029050,0.054306,0.007622,-0.003262,-0.111484,-0.010478,0.049115,0.019468,-0.138843,-0.071557,-0.044055,0.059621,0.028545,2,"[-0.11269804, 0.041881617, -0.02640572, 0.0012..."
26,1,this why ocean health important all the extra ...,-0.112885,0.037529,-0.032567,-0.005534,-0.013056,-0.213296,-0.060226,-0.043620,-0.144942,0.057932,0.028997,-0.153073,-0.022869,-0.039040,0.037722,-0.027484,0.066352,-0.046662,0.167570,-0.061866,-0.031219,0.068458,-0.022404,0.081685,0.033430,-0.010514,0.004991,0.028224,-0.073275,-0.018913,-0

In [ ]:
cluster_two['label'].value_counts()
#Cluster 2 = Cluster Enabling Context

1    100
0     82
3     74
2     12
4     11
Name: label, dtype: int64

**Penentuan Cluster Three**

In [ ]:
cluster_three = df_final.loc[df_final['cluster'] == 3]
cluster_three

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
4,0,predicts that the current rate there likely in...,-0.102229,0.067026,0.019315,0.038599,-0.027093,-0.219828,-0.011722,0.001191,-0.102072,0.089824,-0.006155,-0.102504,-0.031258,-0.031609,0.039617,-0.053095,0.118546,-0.058241,0.124888,-0.056428,-0.015225,0.049737,0.006300,0.034104,-0.002576,-0.041102,-0.019003,0.031235,-0.016504,-0.003390,-0.025735,-0.107479,-0.009990,0.024677,-0.067040,0.023947,-0.045911,-0.021708,...,0.065883,0.000475,0.007120,-0.036393,0.043222,-0.025028,-0.017490,0.042263,-0.064573,-0.115126,0.019282,-0.020778,-0.034549,-0.030730,-0.032842,-0.094107,-0.093373,0.002244,-0.123450,0.048515,0.049222,-0.059212,0.010422,-0.090393,-0.076917,0.047555,0.081577,0.007319,-0.039282,-0.053014,0.016602,0.024164,0.035836,-0.099633,-0.077561,-0.022484,0.073114,0.000308,3,"[-0.10183591, 0.033912547, -0.017202158, 0.005..."
5,0,warming the ocean affects marine species cause...,-0.110090,0.043585,-0.053537,-0.001885,-0.016721,-0.168567,-0.046250,-0.010012,-0.093502,0.068475,0.033313,-0.103038,0.011115,-0.020432,0.024365,-0.026875,0.054176,-0.013432,0.136993,-0.016489,-0.021663,0.017812,-0.002392,0.041823,-0.021681,-0.020769,-0.006857,0.022273,-0.029160,0.024933,-0.003953,-0.107985,0.033184,0.012270,-0.047860,-0.016912,-0.046124,-0.008407,...,0.040960,0.002209,-0.020552,-0.007455,0.097814,-0.008563,-0.017574,0.029337,-0.030562,-0.036281,0.009421,-0.006088,-0.043246,-0.009023,-0.056127,-0.041752,-0.022015,0.033693,-0.116770,0.037576,0.062914,-0.019388,0.017896,-0.090579,-0.069256,0.024723,0.091636,0.019425,-0.028204,-0.066793,0.008347,-0.019255,0.022219,-0.088734,-0.045005,-0.011104,0.077569,0.041563,3,"[-0.10183591, 0.033912547, -0.017202158, 0.005..."
14,3,owe our children separate atmospheric climate ...,-0.106440,0.049548,-0.028496,-0.011476,-0.016493,-0.190770,-0.029531,-0.044049,-0.117460,0.083722,-0.012054,-0.129349,-0.031386,-0.025311,0.011115,-0.053113,0.093352,-0.067904,0.188209,-0.077482,0.013774,0.054491,-0.019155,0.077339,0.007362,0.014455,-0.015158,0.016498,-0.098529,-0.012237,-0.002966,-0.160345,-0.001877,-0.011577,-0.051283,0.063000,-0.055390,-0.068093,...,0.068905,-0.043716,0.011959,-0.030733,0.074910,-0.015740,-0.005659,0.044633,-0.069711,-0.060904,0.021247,-0.040367,-0.054783,-0.012075,-0.049533,-0.078907,-0.077555,-0.043660,-0.094342,0.050182,0.048625,-0.056595,-0.002814,-0.153534,-0.073457,0.021126,0.055700,-0.015799,0.073538,-0.064584,0.000465,0.018123,0.048298,-0.130858,-0.068987,-0.019647,0.066650,0.034344,3,"[-0.10183591, 0.033912547, -0.017202158, 0.005..."
20,1,are víctor hugo hernández eliárraga víctor can...,-0.024917,0.057272,0.018060,0.062737,-0.010659,-0.167408,-0.023890,0.002028,-0.072458,0.081155,0.009438,-0.086620,-0.008704,-0.020524,0.015208,-0.009159,0.094228,-0.039108,0.099138,-0.023202,-0.016869,0.035733,-0.051513,0.037385,0.031100,-0.023140,0.003819,0.054097,-0.025569,0.049014,-0.045278,-0.075991,-0.001660,-0.001633,-0.052465,0.050059,-0.010121,0.016592,...,0.093153,0.002531,0.000716,0.007899,0.064801,-0.019608,0.005322,0.023641,-0.042664,-0.081493,0.003700,-0.016611,-0.027591,-0.036284,-0.030162,-0.074032,-0.043471,0.052482,-0.096498,0.023340,0.050711,-0.034882,0.013403,-0.026034,-0.059525,-0.013842,0.059624,0.043765,0.034505,-0.094604,-0.013483,-0.020358,-0.009561,-0.074286,0.004473,-0.016255,-0.012896,-0.006293,3,"[-0.10183591, 0.033912547, -0.017202158, 0.005..."
27,0,octopus vulgaris the not common octopus irish ...,-0.122392,0.034041,-0.009210,0.024072,-0.012802,-0.209638,-0.049844,-0.001920,-0.099000,0.058037,0.004958,-0.146141,0.024312,0.005766,0.042908,-0.040264,0.086937,-0.028626,0.170157,-0.033910,0.017424,0.011208,-0.012890,0.065119,0.001373,0.006483,0.016319,0.056082,-0.065933,0.006154,-0.0

In [ ]:
cluster_three['label'].value_counts()
#Cluster 3 = Cluster Desirability

0    76
1    55
3    43
2     9
4     6
Name: label, dtype: int64

**Penentuan Cluster Four**

In [ ]:
cluster_four = df_final.loc[df_final['cluster'] == 4]
cluster_four

,label,text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,cluster,centroid
9,1,ban that contains and octinoxate went into eff...,-0.086234,0.057049,0.052829,0.053444,-0.045489,-0.179489,-0.048075,-0.006554,-0.091912,0.041428,0.005415,-0.139263,-0.038025,-0.056052,-0.009616,0.009246,0.114466,-0.050364,0.086993,-0.056034,0.018714,-0.002827,0.011067,0.093867,-0.017628,-0.042467,0.020389,-0.026534,0.045401,0.037845,-0.003214,-0.088164,-0.006665,-0.017912,-0.076651,0.044219,-0.027934,0.048554,...,0.036015,0.005842,-0.002240,0.031122,0.063729,-0.035785,0.019353,0.032317,0.015394,-0.079033,-0.015942,-0.021296,-0.039428,-0.045764,-0.056441,-0.093469,-0.050919,0.036038,-0.077858,0.107161,0.085022,-0.029322,0.008904,-0.076545,-0.050143,0.008110,0.105836,-0.004431,0.044172,-0.098106,-0.008612,0.014689,0.038539,-0.050599,-0.055653,-0.008765,0.056121,0.089633,4,"[-0.09115943, 0.03382597, -0.0011490898, 0.018..."
81,1,that the loss ice over years another part the ...,-0.104644,0.048016,-0.002681,-0.009576,-0.017593,-0.203856,-0.032830,-0.029200,-0.116397,0.044684,0.048234,-0.106592,-0.026717,-0.022806,0.056170,-0.075619,0.080077,-0.044362,0.152496,-0.023055,0.011509,0.063845,-0.005528,0.024551,-0.027107,-0.015217,-0.030252,0.055044,-0.048301,-0.012504,-0.023454,-0.123817,0.029913,0.033341,-0.115394,0.039597,-0.026439,0.019449,...,0.068866,0.028013,0.032938,-0.040175,0.071779,0.020676,0.026692,0.036686,-0.087142,-0.064622,0.005136,-0.108569,-0.016931,0.015128,-0.012657,-0.094452,-0.018460,0.031836,-0.092984,0.031835,0.088217,-0.032168,0.009113,-0.100949,-0.031415,0.007550,0.098700,0.005472,-0.009607,-0.056956,-0.002859,0.005658,0.063077,-0.101196,-0.012148,0.007585,0.105009,0.013035,4,"[-0.09115943, 0.03382597, -0.0011490898, 0.018..."
105,1,already knew that ocean warming the main facto...,-0.094732,0.011300,-0.028891,0.014554,-0.010455,-0.173031,-0.049330,-0.014610,-0.074005,0.054003,-0.010848,-0.140913,-0.001005,-0.024956,0.036750,-0.060337,0.053275,-0.049102,0.122444,-0.049774,-0.088981,0.071059,-0.011957,0.043028,0.013920,-0.019650,0.013216,0.037411,-0.073609,-0.037851,0.027091,-0.111915,0.041689,0.021036,-0.061368,0.047818,-0.055025,-0.066229,...,0.084822,0.016378,0.027738,-0.029519,0.052890,-0.021515,-0.049858,0.055477,-0.015137,-0.055675,0.010863,-0.063594,-0.082580,0.006928,-0.040757,-0.085859,-0.047583,0.015527,-0.108917,-0.001900,0.064198,-0.015968,-0.000310,-0.098408,-0.073431,-0.009784,0.056032,0.002561,0.003217,-0.071034,-0.009593,-0.002944,0.002030,-0.054806,-0.050983,0.004444,0.051429,0.037720,4,"[-0.09115943, 0.03382597, -0.0011490898, 0.018..."
131,0,things are not for one coral bleaching begins ...,-0.086693,0.015122,0.014143,0.023276,-0.024835,-0.154091,-0.006987,0.015481,-0.111535,0.057962,0.021293,-0.111915,0.003532,-0.012753,0.037749,-0.007553,0.071638,-0.035533,0.138730,-0.047140,0.015698,0.056273,0.004683,0.097594,-0.013161,0.031663,-0.013893,0.000575,-0.071648,0.021071,-0.003644,-0.121672,0.031460,0.006877,-0.054092,0.037739,-0.037899,0.016785,...,0.022822,-0.025388,0.050054,0.028960,0.048160,-0.028829,0.026347,0.050862,-0.035022,-0.036524,0.002047,-0.006784,-0.076181,-0.044852,-0.053027,-0.021692,-0.055927,-0.002265,-0.103106,0.034061,0.122343,-0.062488,0.021439,-0.090044,-0.028299,-0.023450,0.055381,0.004409,-0.020943,-0.077419,0.008615,-0.018096,0.055067,-0.091323,-0.040487,0.013927,0.094334,0.011774,4,"[-0.09115943, 0.03382597, -0.0011490898, 0.018..."
153,1,moreso oceans are not really option here for t...,-0.116218,0.044962,0.003545,0.058791,0.010360,-0.180303,-0.017843,0.020240,-0.072265,0.048750,0.010309,-0.118214,-0.042885,-0.034813,0.033622,-0.002368,0.049639,-0.022016,0.105638,-0.022248,-0.010551,0.023037,0.067433,0.083581,-0.021555,0.012243,-0.032303,0.005405,-0.040052,0.008881,0.043187,-0.0989

In [ ]:
cluster_four['label'].value_counts()
#Cluster 4 = Cluster Desirability

0    21
1    14
3    10
2     5
4     2
Name: label, dtype: int64